In [1]:
import numpy as np
import random

In [2]:
def print_sample(sample_ix, ix_to_char):
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    txt = txt[0].upper() + txt[1:]  # capitalize first character 
    print ('%s' % (txt, ), end='')
def smooth(loss, cur_loss):
    return loss * 0.999 + cur_loss * 0.001
def get_initial_loss(vocab_size, seq_length):
    return -np.log(1.0/vocab_size)*seq_length
def softmax(x):
    x=x-np.max(x)
    return np.exp(x)/np.sum(np.exp(x),axis=0)
def rnn_step_forward(parameters, a_prev, x):
    Waa, Wax, Wya, by, b = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['b']
    a_next = np.tanh(np.dot(Wax, x) + np.dot(Waa, a_prev) + b) # hidden state
    p_t = softmax(np.dot(Wya, a_next) + by) # unnormalized log probabilities for next chars # probabilities for next chars 
    return a_next, p_t
def rnn_step_backward(dy, gradients, parameters, x, a, a_prev):
    gradients['dWya'] += np.dot(dy, a.T)
    gradients['dby'] += dy
    da = np.dot(parameters['Wya'].T, dy) + gradients['da_next'] # backprop into h
    daraw = (1 - a * a) * da # backprop through tanh nonlinearity
    gradients['db'] += daraw
    gradients['dWax'] += np.dot(daraw, x.T)
    gradients['dWaa'] += np.dot(daraw, a_prev.T)
    gradients['da_next'] = np.dot(parameters['Waa'].T, daraw)
    return gradients
def rnn_forward(X, Y, a0, parameters, vocab_size = 27):
    x, a, y_hat = {}, {}, {}
    a[-1] = np.copy(a0)
    loss = 0
    for t in range(len(X)):
        x[t] = np.zeros((vocab_size,1)) 
        if (X[t] != None):
            x[t][X[t]] = 1
        a[t], y_hat[t] = rnn_step_forward(parameters, a[t-1], x[t])
        loss -= np.log(y_hat[t][Y[t],0]) 
    cache = (y_hat, a, x)
    return loss, cache
def rnn_backward(X, Y, parameters, cache):
    gradients = {}
    (y_hat, a, x) = cache
    Waa, Wax, Wya, by, b = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['b']
    gradients['dWax'], gradients['dWaa'], gradients['dWya'] = np.zeros_like(Wax), np.zeros_like(Waa), np.zeros_like(Wya)
    gradients['db'], gradients['dby'] = np.zeros_like(b), np.zeros_like(by)
    gradients['da_next'] = np.zeros_like(a[0])
    for t in reversed(range(len(X))):
        dy = np.copy(y_hat[t])
        dy[Y[t]] -= 1
        gradients = rnn_step_backward(dy, gradients, parameters, x[t], a[t], a[t-1])
    return gradients, a
def update_parameters(parameters, gradients, lr):
    parameters['Wax'] += -lr * gradients['dWax']
    parameters['Waa'] += -lr * gradients['dWaa']
    parameters['Wya'] += -lr * gradients['dWya']
    parameters['b']  += -lr * gradients['db']
    parameters['by']  += -lr * gradients['dby']
    return parameters
def initialize_parameters(n_a, n_x, n_y):
    np.random.seed(1)
    Wax = np.random.randn(n_a, n_x)*0.01 # input to hidden
    Waa = np.random.randn(n_a, n_a)*0.01 # hidden to hidden
    Wya = np.random.randn(n_y, n_a)*0.01 # hidden to output
    b = np.zeros((n_a, 1)) # hidden bias
    by = np.zeros((n_y, 1)) # output bias
    parameters = {"Wax": Wax, "Waa": Waa, "Wya": Wya, "b": b,"by": by}
    return parameters

In [3]:
data = open('dinos.txt', 'r').read()
data= data.lower()
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('There are %d total characters and %d unique characters in your data.' % (data_size, vocab_size))
with open("dinos.txt") as f:
    examples = f.readlines()
print ("there are %d examples of dino names" % (len(examples)))

There are 19909 total characters and 27 unique characters in your data.
there are 1536 examples of dino names


In [4]:
char_to_ix = { ch:i for i,ch in enumerate(sorted(chars)) }
ix_to_char = { i:ch for i,ch in enumerate(sorted(chars)) }
print(ix_to_char)
print(char_to_ix)

{0: '\n', 1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z'}
{'\n': 0, 'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26}


In [5]:
def clip(gradients, maxValue):
    dWaa = gradients['dWaa']
    dWax = gradients['dWax']
    dWya=gradients['dWya']
    db = gradients['db']
    dby = gradients['dby']
    for gradient in [dWax, dWaa, dWya, db, dby]:
        np.clip(gradient,-maxValue,maxValue,out=gradient)
    gradients = {"dWaa": dWaa, "dWax": dWax, "dWya": dWya, "db": db, "dby": dby}
    return gradients

In [6]:
def sample(parameters, char_to_ix):
    Waa, Wax, Wya, by, b = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['b']
    vocab_size = by.shape[0]
    n_a = Waa.shape[1]
    x = np.zeros((vocab_size, 1)) #initialize
    a_prev = np.zeros((n_a, 1))  #initialize
    indices = []
    idx = -1 # detect newline char
    counter = 0
    newline_character = char_to_ix['\n']
    #print ("char_to_ix['\n']", char_to_ix['\n'])
    while (idx != newline_character and counter != 50): # stop if 50 chars sampled w/o EOS/"\n"
        a = np.tanh(np.matmul(Wax,x) + np.matmul(Waa,a_prev) + b)
        y = softmax(np.matmul(Wya,a) + by)
        idx = np.random.choice(list(range(vocab_size)), p = y.ravel()) # sample. 
        indices.append(idx)
        x = np.zeros((vocab_size, 1))
        x[idx] = 1
        a_prev = a
        counter +=1
    if (counter == 50):
        indices.append(char_to_ix['\n'])
    #print ("counter :",counter)
    return indices

In [7]:
def optimize(X, Y, a_prev, parameters, learning_rate = 0.01):
    """
    Returns:
    loss -- value of the loss function (cross-entropy)
    gradients -- python dictionary containing:
                        dWax -- Gradients of input-to-hidden weights, of shape (n_a, n_x)
                        dWaa -- Gradients of hidden-to-hidden weights, of shape (n_a, n_a)
                        dWya -- Gradients of hidden-to-output weights, of shape (n_y, n_a)
                        db -- Gradients of bias vector, of shape (n_a, 1)
                        dby -- Gradients of output bias vector, of shape (n_y, 1)
    a[len(X)-1] -- the last hidden state, of shape (n_a, 1)
    """
    loss, cache = rnn_forward(X, Y, a_prev, parameters)
    gradients, a = rnn_backward(X, Y, parameters, cache)
    gradients = clip(gradients, 5)
    parameters = update_parameters(parameters, gradients, learning_rate)
    return loss, gradients, a[len(X)-1]

In [8]:
with open("dinos.txt") as f:
    examples = f.readlines()
examples = [x.lower().strip() for x in examples]


In [9]:
def model(data, ix_to_char, char_to_ix, num_iterations = 35000, n_a = 50, dino_names = 7, vocab_size = 27):
    n_x, n_y = vocab_size, vocab_size
    parameters = initialize_parameters(n_a, n_x, n_y)
    loss = get_initial_loss(vocab_size, dino_names)
    with open("dinos.txt") as f:
        examples = f.readlines()
    examples = [x.lower().strip() for x in examples]
    np.random.seed(0)
    np.random.shuffle(examples)
    a_prev = np.zeros((n_a, 1))
    for j in range(num_iterations):
        index = j % len(examples)
        X = [None] + [char_to_ix[ch] for ch in examples[index]] 
        Y = X[1:] + [char_to_ix["\n"]]
        curr_loss, gradients, a_prev = optimize(X, Y, a_prev, parameters, learning_rate = 0.01)
        loss = smooth(loss, curr_loss)
        if j % 2000 == 0:
            print('Iteration: %d, Loss: %f' % (j, loss) + '\n')
            seed = 0
            for name in range(dino_names):
                sampled_indices = sample(parameters, char_to_ix)
                print_sample(sampled_indices, ix_to_char)
                seed += 1   
            print('\n')
    return parameters

In [10]:
parameters = model(data, ix_to_char, char_to_ix,num_iterations = 350000)

Iteration: 0, Loss: 23.087336

Co
Eyhqgyshubjerbjgmis
Qfguyhvbqtdkxb
Okuxhjmozrjgycsxqkeisqywvxgijorg
Jhywwqinubuavuapmdctozzzcyflteoebci
Kqjbagdzduoomziuojypckurxgtloyzwiwqk
Tmutuhunlr


Iteration: 2000, Loss: 27.884160

Amycgangwsacrusycripsiurgtorandprusayrabhustactgny
Honqmeleprusamglonggpodanrosennnsteroshuceranelocr
Inlonaleonxyorytonganytabtonosaunus
Padeleloypurapfnorusaurus
Aurus
Itreriptixtisgurros
Ontepokosauluspmurinontyaicallpeosaurus


Iteration: 4000, Loss: 25.901815

Amkorosaurus
Stdoosaurus
Rusaurus
Eschuszuratanstipanos
Chmmoop
Alorosaurus
Azarosaurus


Iteration: 6000, Loss: 24.608779

Crlhux
Raptorfirus
Deliosaurus
Hua
Fhidosaurushs
S
Eheosaurus


Iteration: 8000, Loss: 24.070350

Loninitan
Sartosaurus
Alicisaurus
Rapistos
Hingiung
Tamethosmilus
Cerapochantoensaurus


Iteration: 10000, Loss: 23.844446

Ceriondestes
Taraxosaurus
Ceniaschyatandomos
Sitorong
Xealochus
Wuronarys
Brovoradis


Iteration: 12000, Loss: 23.291971

Tonoshapuggosaurus
Gkuenotes
Meginsthuscenod

Iteration: 124000, Loss: 22.135875

Ajbicophylia
Khemaniminytherytosaurus
Hosaurus
Releykala
Trhecopelta
Trivecsaurus
Bracochisurirovessaurus


Iteration: 126000, Loss: 22.192819

Rophongosaurus
Asiceratops
Sinoceratops
Wochyosolongosaurus
Wuinasaurus
Hozhatocaenitimus
Isaurus


Iteration: 128000, Loss: 22.074889

Hinolophia
Reschinqusaurus
Orrotongoraxton
Euctelorasaurus
Yenchungoz
Shuelosaurus
Ilorovleuilgeicisaurus


Iteration: 130000, Loss: 21.756508

Xunhacruanchon
Yusaurus
Yuanosaurus
Huaraxacerathiaodrnissclophanventa
Eveosaurus
Inacoranosaurus
Mazhavenathua


Iteration: 132000, Loss: 21.841045

Artachyinitesisaurus
Ianpolisaurus
Begirachitmisaurus
Taciangohasaurus
Icalonognath
Iciainarhansaurus
Wgrsaurus


Iteration: 134000, Loss: 22.068716

Oson
Racomo
Enithomakian
Igisaurus
Inisaurus
Eirtangomasaudus
Wongachenade


Iteration: 136000, Loss: 21.995873

Leehosaurus
Eustenithobaranhsa
Eitha
Erdosichianlosaurus
Et
Palaur
Pellnasaptos


Iteration: 138000, Loss: 21.939324

Inosaurus

Iteration: 248000, Loss: 24.153512

Chiyosaurus
Cininengosurkigoratops
Janoloularosauritlshtekgosaurosaurus
Ilosaurus
Arilornanygnasiphorgnypestecongropsshaepritus
Caungysus
Eniosauranigpakptagossaurus


Iteration: 250000, Loss: 23.983467

Cilniaristinostodoatus
Ehinesaurus
Calitrhus
Ihnoteratops
Chyganosaurus
Iiasaurus
Chitopherampeeikisis


Iteration: 252000, Loss: 24.116386

Chasaurus
Ieusodon
Camadtodon
Iyasaurus
Cermus
Chiiuabadoa
Chmndosaurus


Iteration: 254000, Loss: 24.062341

Craxosaurus
Chuasaurus
Cermasiphosaurus
Riatrosaurus
Erosenosaurus
Clanamamphinolatitmodongocilosaurus
Ceicirus


Iteration: 256000, Loss: 23.872782

Caceramyis
Ehiasaurus
Caeassmonosaurus
Ryumia
Iania
Cean
Cacoramus


Iteration: 258000, Loss: 24.015353

Ceucasacrus
Rorenaosaurus
Chesaurus
Riganaptosaurus
Chanianhesesaurus
Changanoceratops
Canbasaurus


Iteration: 260000, Loss: 24.066434

Canisaurus
Chebais
Cuangectakkomaptan
Ehimurrechriskannyus
Chyznoysaurus
Chus
Camamanstuyctergaprocacechuasaurus


It